In [84]:
%matplotlib inline

In [72]:
import time
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix as confu
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import grid_search
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.cross_validation import cross_val_score

In [55]:
training = pd.read_csv('kdd_10_per.csv')
testing= pd.read_csv('corrected', header=None)
attack_list = open('training_attack_types.txt')
attack_dic = {}

for line in attack_list.readlines():    
    detail = line.split(' ')
    if len(detail) == 1:
        break
    attack_type = detail[1][:-1]
    attack_name = detail[0]
    
    if attack_dic.has_key(attack_type):
        attack_dic[attack_type].append(attack_name)
    else:
        attack_dic[attack_type] = []
        attack_dic[attack_type].append(attack_name)

string_label_columns = ['flag','service','protocol_type'] 
testing.columns = training.columns
#for i in range(len(string_label_columns)):
#    del(training[string_label_columns[i]])

In [56]:
test_attack_dic = {}
test_attack_dic["apache2."] = 2 
test_attack_dic["back."] = 2 
test_attack_dic["buffer_overflow."] = 3 
test_attack_dic["ftp_write."] = 4 
test_attack_dic["guess_passwd."] = 4 
#test_attack_dic["httptunnel."] = 4 
test_attack_dic["httptunnel."] = 3 
test_attack_dic["imap."] = 4 
test_attack_dic["ipsweep."] = 1 
test_attack_dic["land."] = 2 
test_attack_dic["loadmodule."] = 3 
test_attack_dic["mailbomb."] = 2 
test_attack_dic["mscan."] = 1 
test_attack_dic["multihop."] = 4 
#test_attack_dic["multihop."] = 3 # note that this is a duplicate 
test_attack_dic["named."] = 4 
test_attack_dic["neptune."] = 2 
test_attack_dic["nmap."] = 1 
test_attack_dic["perl."] = 3 
test_attack_dic["phf."] = 4 
test_attack_dic["pod."] = 2 
test_attack_dic["portsweep."] = 1 
test_attack_dic["processtable."] = 2 
test_attack_dic["ps."] = 3 
test_attack_dic["rootkit."] = 3 
test_attack_dic["saint."] = 1 
test_attack_dic["satan."] = 1 
test_attack_dic["sendmail."] = 4 
test_attack_dic["smurf."] = 2 
test_attack_dic["snmpgetattack."] = 4 
test_attack_dic["snmpguess."] = 4 
test_attack_dic["sqlattack."] = 3 
test_attack_dic["teardrop."] = 2 
test_attack_dic["udpstorm."] = 2 
test_attack_dic["warezmaster."] = 4 
test_attack_dic["worm."] = 4 
test_attack_dic["xlock."] = 4 
test_attack_dic["xsnoop."] = 4 
test_attack_dic["xterm."] = 3
test_attack_dic['normal.'] = 0
attack_dic_num = {}
attack_dic_num[0] = ['normal']
attack_dic_num[2] = attack_dic['dos']
attack_dic_num[4] = attack_dic['r2l']
attack_dic_num[3] = attack_dic['u2r']
attack_dic_num[1] = attack_dic['probe']

In [58]:
#anomaly category
category_label_binary = []
map(lambda label: map(lambda cate: category_label_binary.append(cate) if label in attack_dic_num[cate] else None,attack_dic_num.keys()),training['label'])
category_label_binary = np.array(category_label_binary)
category_label_binary[np.where((category_label_binary != 0))] = 1

print 'done'
test_category_label_binary = []
map(lambda label: test_category_label_binary.append(test_attack_dic[label]),testing['label'])
test_category_label_binary = np.array(test_category_label_binary)
test_category_label_binary[np.where((test_category_label_binary != 0))] = 1

done


In [43]:
#attack category
category_label = []
map(lambda label: map(lambda cate: category_label.append(cate) if label in attack_dic_num[cate] else None,attack_dic_num.keys()),training['label'])
category_label = np.array(category_label)
#category_label[np.where((category_label != 0))] = 1

print 'done'
test_category_label = []
map(lambda label: test_category_label.append(test_attack_dic[label]),testing['label'])
test_category_label = np.array(test_category_label)
#test_category_label[np.where((test_category_label != 0))] = 1

done


In [62]:
len(testing_index_array[0])

291440

In [ ]:
#anomaly index
training_index_array = []
testing_index_array = []
for j in range(3,30):
    training_index = []
    testing_index = []
    for name in ['dos', 'u2r', 'r2l', 'probe']:
        now_attack_type = attack_dic[name]
        now_attack_type_dic = {'normal':0}
        for i in range(len(now_attack_type)):
            now_attack_type_dic[now_attack_type[i]] = i+1
    
        for now_name in now_attack_type:
            #print len(np.where(training['label'] == now_name)[0].tolist())
            now_attack_index = np.where(training['label'] == now_name)[0]
            attack_len = len(now_attack_index)
            
            if attack_len > (1000*j):  
                #print attack_len     
                now_random_index = now_attack_index[np.random.choice(attack_len, j*1000, replace=False)]
                now_unrepeat_index = list(set(now_attack_index) - set(now_random_index))
                training_index = training_index + now_random_index.tolist()
                if now_unrepeat_index <= pow(10,5):
                    testing_index = testing_index + now_unrepeat_index
                else:
                    testing_index = testing_index + now_unrepeat_index[:pow(10,5)]
                                 
            else:
                now_random_index = now_attack_index[np.random.choice(attack_len, int(0.5*attack_len), replace=False)]
                now_unrepeat_index = list(set(now_attack_index) - set(now_random_index))                
                training_index = training_index + now_random_index.tolist()                
                testing_index = testing_index + now_unrepeat_index
            

    training_index = training_index + np.where(category_label_binary == 0)[0][:int(0.5*len(training_index))].tolist()    
    training_index_array.append(training_index)
    testing_index = testing_index + np.where(category_label_binary == 0)[0][int(0.05*len(testing_index)):int(1.05*len(testing_index))].tolist() 
    testing_index_array.append(testing_index)




In [ ]:
#attack index
training_attack_index_array = []
for i in range(3,15):
    training_index = []
    for name in ['dos', 'u2r', 'r2l', 'probe']:   
        now_attack_type = attack_dic[name]
        now_attack_type_dic = {'normal':0}
        for j in range(len(now_attack_type)):
            now_attack_type_dic[now_attack_type[j]] = j+1
    
        for now_name in now_attack_type:
            attack_len = len(np.where(training['label'] == now_name)[0].tolist())
            #print attack_len, attack_len > (i*10000)
            if attack_len > (i*1000):  
        
                training_index = training_index + np.where(training['label'] == now_name)[0][np.random.randint(0,attack_len, i*1000)].tolist()
            else:
                training_index = training_index + np.where(training['label'] == now_name)[0].tolist()
           
    training_attack_index_array.append(training_index)

testing_index = np.where(test_category_label != 0)[0].tolist() 
training_key_index = np.where(category_label != 0)[0].tolist()
#training_index = training_index + np.where(category_label == 0)[0][:len(training_index)*1].tolist()
#testing_index = np.where(test_category_label == 1)[0].tolist() + np.where(test_category_label == 2)[0][:10000].tolist() \
#                + np.where(test_category_label == 3)[0].tolist() + np.where(test_category_label == 4)[0][:5000].tolist()

# Combine the string columns

In [60]:
#anomaly key
str_combination_anomaly = []
map(lambda x: str_combination_anomaly.append((x[0]+'_'+x[1]+'_'+x[2])) if (x[0]+'_'+x[1]+'_'+x[2]) not in str_combination_anomaly else None,training[string_label_columns].values)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
#attack type key
str_combination_attack = []
map(lambda x: str_combination_attack.append((x[0]+'_'+x[1]+'_'+x[2])) if (x[0]+'_'+x[1]+'_'+x[2]) not in str_combination_attack else None,training.iloc[training_key_index][string_label_columns].values)

In [61]:
#anomaly key sort
pro_ser_flag_count_anomaly = {}
for x in str_combination_anomaly:
    pro_ser_flag_count_anomaly[x] = 0
for line in training[string_label_columns].values:
    key = line[0]+'_'+line[1]+'_'+line[2]
    pro_ser_flag_count_anomaly[key] = pro_ser_flag_count_anomaly[key] + 1

In [ ]:
#attack key sort
pro_ser_flag_count_attack = {}
for x in str_combination_attack:
    pro_ser_flag_count_attack[x] = 0
for line in training.iloc[training_key_index][string_label_columns].values:
    key = line[0]+'_'+line[1]+'_'+line[2]
    pro_ser_flag_count_attack[key] = pro_ser_flag_count_attack[key] + 1

In [95]:
#anomlay top key 
# & (len(new_string_columns_anomaly) < 100)
sorted_dic_anomaly = sorted(pro_ser_flag_count_anomaly.iteritems(), key=lambda (k,v): (v,k),reverse=True)
new_string_columns_anomaly = []
new_string_columns_dic_anomaly = {}
loc_value = 1
new_string_columns_anomaly.append('others')
new_string_columns_dic_anomaly['others'] = 0
for string in sorted_dic_anomaly:
    if (string[1] > 0) :
        new_string_columns_anomaly.append(string[0])
        new_string_columns_dic_anomaly[string[0]] = loc_value
        loc_value = loc_value+1
    else:       
        break
inv_new_string_columns_dic_anomaly = {v: k for k, v in new_string_columns_dic_anomaly.items()}

In [ ]:
#attack top key 
sorted_dic_attack = sorted(pro_ser_flag_count_attack.iteritems(), key=lambda (k,v): (v,k),reverse=True)
new_string_columns_attack = []
new_string_columns_dic_attack = {}
loc_value = 0
for string in sorted_dic_attack:
    if (string[1] > 1) & (len(new_string_columns_attack) < 100):
        new_string_columns_attack.append(string[0])
        new_string_columns_dic_attack[string[0]] = loc_value
        loc_value = loc_value+1
    else:
        new_string_columns_attack.append('others')
        new_string_columns_dic_attack['others'] = loc_value
        break
inv_new_string_columns_dic_attack = {v: k for k, v in new_string_columns_dic_attack.items()}

# Creating columns

In [96]:
#anomaly column
training_anomaly_copy = training.copy()
testing_anomaly_copy = testing.copy()
for column_name in new_string_columns_anomaly:
    training_anomaly_copy[column_name] = np.zeros(len(training))
    testing_anomaly_copy[column_name] = np.zeros(len(testing))
string_vector_anomaly = np.zeros([len(training_anomaly_copy),len(new_string_columns_anomaly)])
testing_string_vector_anomlay = np.zeros([len(testing_anomaly_copy),len(new_string_columns_anomaly)])
string_vector_loc_anomaly = []
testing_string_vector_loc_anomaly = []
map(lambda x: string_vector_loc_anomaly.append(new_string_columns_dic_anomaly[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_anomaly \
    else string_vector_loc_anomaly.append(new_string_columns_dic_anomaly['others']) ,training_anomaly_copy[string_label_columns].values)
map(lambda x: testing_string_vector_loc_anomaly.append(new_string_columns_dic_anomaly[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_anomaly \
    else testing_string_vector_loc_anomaly.append(new_string_columns_dic_anomaly['others']) ,testing_anomaly_copy[string_label_columns].values)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [39]:
#attack column
training_attack_copy = training.copy()
testing_attack_copy = testing.copy()
for column_name in new_string_columns_attack:
    training_attack_copy[column_name] = np.zeros(len(training))
    testing_attack_copy[column_name] = np.zeros(len(testing))
string_vector_attack = np.zeros([len(training_attack_copy),len(new_string_columns_attack)])
testing_string_vector_attack = np.zeros([len(testing_attack_copy),len(new_string_columns_attack)])
string_vector_loc_attack = []
testing_string_vector_loc_attack = []
map(lambda x: string_vector_loc_attack.append(new_string_columns_dic_attack[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_attack \
    else string_vector_loc_attack.append(new_string_columns_dic_attack['others']) ,training_attack_copy[string_label_columns].values)
map(lambda x: testing_string_vector_loc_attack.append(new_string_columns_dic_attack[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_dic_attack \
    else testing_string_vector_loc_attack.append(new_string_columns_dic_attack['others']) ,testing_attack_copy[string_label_columns].values)

NameError: name 'new_string_columns_attack' is not defined

In [84]:
training.columns

Index([u'duration', u'protocol_type', u'service', u'flag', u'src_bytes',
       u'dst_bytes', u'land', u'wrong_fragment', u'urgent', u'hot',
       u'num_failed_logins', u'logged_in', u'num_compromised', u'root_shell',
       u'su_attempted', u'num_root', u'num_file_creations', u'num_shells',
       u'num_access_files', u'num_outbound_cmds', u'is_host_login',
       u'is_guest_login', u'count', u'srv_count', u'serror_rate',
       u'srv_serror_rate', u'rerror_rate', u'srv_rerror_rate',
       u'same_srv_rate', u'diff_srv_rate', u'srv_diff_host_rate',
       u'dst_host_count', u'dst_host_srv_count', u'dst_host_same_srv_rate',
       u'dst_host_diff_srv_rate', u'dst_host_same_src_port_rate',
       u'dst_host_srv_diff_host_rate', u'dst_host_serror_rate',
       u'dst_host_srv_serror_rate', u'dst_host_rerror_rate',
       u'dst_host_srv_rerror_rate', u'label'],
      dtype='object')

In [64]:
def replace_value(training_normal_numpy, key, loc):
    training_normal_numpy[loc][key+38] = 1
   

In [85]:
training_anomaly_copy = training.copy()
testing_anomaly_copy = testing.copy()
for column_name in string_label_columns: 
    print column_name
    training_anomaly_copy = training_anomaly_copy.join(pd.get_dummies(training_anomaly_copy[column_name]))
    

flag
service
protocol_type


In [97]:
#anomaly delete
#training_anomaly_copy = training.copy()
#testing_anomaly_copy = testing.copy()
#print training_anomaly_copy.columns, len(training_anomaly_copy.iloc[0])
for i in range(len(string_label_columns)):   
    del(training_anomaly_copy[string_label_columns[i]])
    del(testing_anomaly_copy[string_label_columns[i]])
del training_anomaly_copy['label']
del testing_anomaly_copy['label']


In [ ]:
#attack delete
for i in range(len(string_label_columns)):
    del(training_attack_copy[string_label_columns[i]])
    del(testing_attack_copy[string_label_columns[i]])
del training_attack_copy['label']
del testing_attack_copy['label']

In [99]:
#anomaly replace
training_numpy_anomaly = training_anomaly_copy.values
testing_numpy_anomaly = testing_anomaly_copy.values
map(lambda i: replace_value(training_numpy_anomaly, string_vector_loc_anomaly[i], i),range(len(training_anomaly_copy)))
map(lambda i: replace_value(testing_numpy_anomaly, testing_string_vector_loc_anomaly[i],i),range(len(testing_anomaly_copy)))


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [66]:
#attack replace
training_numpy_attack = training_attack_copy.values
testing_numpy_attack = testing_attack_copy.values
map(lambda i: replace_value(training_numpy_attack, string_vector_loc_attack[i], i),range(len(training_attack_copy)))
map(lambda i: replace_value(testing_numpy_attack, testing_string_vector_loc_attack[i], i),range(len(testing_attack_copy)))

NameError: global name 'string_vector_loc_attack' is not defined

In [100]:
def pca_transform(training_numpy, testing_numpy, index, pca_dim, training_copy):
    
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy[index])
    testting_x_scaled = min_max.transform(testing_numpy)
    training_norm = pd.DataFrame(x_scaled, columns = training_copy.columns)
    testing_norm = pd.DataFrame(testting_x_scaled, columns = training_copy.columns)
    
    pca_model = PCA(n_components=pca_dim)
    training_pca_value = pca_model.fit_transform(training_norm)
    testing_pca_value = pca_model.transform(testing_norm)
    return (training_pca_value, testing_pca_value)

In [90]:
training_numpy_anomaly = training_anomaly_copy.values
testing_numpy_anomaly = testing_anomaly_copy.values

In [108]:
training_index_array[17]

IndexError: list index out of range

In [102]:
#no combination 0.834888827889 
#combination 0.874660307439
pca_result_combine, pca_result_binary 

([(5, 0.93616390232776436),
  (10, 0.94923467541266648),
  (15, 0.92542919512704391),
  (20, 0.94284089847274299),
  (25, 0.93647048938306976),
  (30, 0.93720185237846909),
  (35, 0.93795680306109541),
  (40, 0.93788603165274442),
  (45, 0.9380275828193978)],
 [(5, 0.94258134487882272),
  (10, 0.93977358644981324),
  (15, 0.93569201500237753),
  (20, 0.93927823944571753),
  (25, 0.94041067937945966),
  (30, 0.93795701453039815),
  (35, 0.93871201251935033),
  (40, 0.9389243740533546),
  (45, 0.93913672445518459)])

In [110]:
pca_result_combine

[(5, 0.93848023091932142),
 (10, 0.9471574065404369),
 (15, 0.92694856457333086),
 (20, 0.94403280073383333),
 (25, 0.93861513815752251),
 (30, 0.93904214232451311),
 (35, 0.93991884233575274),
 (40, 0.93991884991481134),
 (45, 0.94027852171429438)]

In [109]:
pca_result_combine =[]
for j in xrange(5, 50, 5):
    n_estimators = 10

    svc = SVC()
    clf = OneVsOneClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
    (training_pca_value, testing_pca_value) = pca_transform(training_numpy_anomaly, training_numpy_anomaly[testing_index_array[16]], training_index_array[16], j, training_anomaly_copy)
    #clf.fit(training_pca_value, category_label_binary[training_index_array[array_index]])
    
    try:        
        score = np.average(cross_val_score(clf, training_pca_value, category_label_binary[training_index_array[16]], cv=5, n_jobs=-1))
        print score
        pca_result_combine.append((j, score))
                

    except ValueError:
        print i, 'broke'

0.938480230919


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

0.94715740654


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

0.926948564573


...

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

0.939918849915


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

0.940278521714


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

In [53]:

all_score = []
whole_model = []
best_score = 0
for array_index in xrange(3,len(training_index)):
    for i in xrange(1,2,1):
        for j in xrange(5, 50, 5):
            n_estimators = 10

            svc = SVC(C=pow(10,i))
            clf = OneVsOneClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
            (training_pca_value, testing_pca_value) = pca_transform(training_numpy_anomaly, training_numpy_anomaly[testing_index_array[array_index]], training_index_array[array_index], j, training_anomaly_copy)
            clf.fit(training_pca_value, category_label_binary[training_index_array[array_index]])
            try:
                
                score = clf.score(testing_pca_value, category_label_binary[testing_index_array[array_index]])
                if score > best_score:
                    print score
                    best_score = score
                    best_param = (array_index, i, j, score)
                

            except ValueError:
                print i, 'broke'

0.874660307439


/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


KeyboardInterrupt: 

In [76]:
#anomaly training 
n_estimators = 10
start = time.time()
svc = SVC(C=pow(10,1))
anomaly_model = OneVsOneClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
(training_pca_value_anomaly, testing_pca_value_anomaly) = pca_transform(training_numpy_anomaly, testing_numpy_anomaly, training_index_array[16],5, training_anomaly_copy)
anomaly_model.fit(training_pca_value_anomaly, category_label_binary[training_index_array[16]])
anomaly_model.score(testing_pca_value_anomaly, test_category_label_binary)
#anomaly_result = anomoly_model.predict(testing_pca_value_anomaly)
#anomaly_index = np.where(anomaly_result == 1)[0].tolist()

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


0.92250883358143454

In [78]:
len(training_index_array)

17

In [77]:
confu(anomaly_model.predict(testing_pca_value_anomaly), test_category_label_binary)

array([[ 54993,  18502],
       [  5600, 231934]])

In [141]:
#anomaly training 
n_estimators = 10
start = time.time()
svc = SVC(C=pow(10,4))
anomoly_model = OneVsOneClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
(training_pca_value_anomaly, testing_pca_value_anomaly) = pca_transform(training_numpy_anomaly, testing_numpy_anomaly, training_anomaly_index_array[0],25, training_anomaly_copy)
anomaly_model.fit(training_pca_value_anomaly, category_label_binary[training_anomaly_index_array[0]])
anomaly_result = anomoly_model.predict(testing_pca_value_anomaly)
anomaly_index = np.where(anomaly_result == 1)[0].tolist()

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


In [130]:
#attack training 
n_estimators = 10
start = time.time()
svc = SVC(C=pow(10,4))
attack_model = OneVsRestClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
(training_pca_value_attack, testing_pca_value_attack) = pca_transform(training_numpy_attack, testing_numpy_attack, training_attack_index_array[8],5, training_attack_copy)
attack_model.fit(training_pca_value_attack, category_label[training_attack_index_array[8]])
anomaly_result[anomaly_index] = attack_model.predict(testing_pca_value_attack[anomaly_index])

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

In [131]:
sum(anomaly_result == test_category_label) / float(len(anomaly_result))

0.93239215635841033

In [132]:
confu(anomaly_result, test_category_label)

array([[ 46015,      5,    284,      7,     83],
       [   636,   3893,    825,    149,    105],
       [  1130,    167, 228256,      5,   3932],
       [   656,      8,    106,     17,    249],
       [ 12156,     93,    382,     50,  11820]])

In [142]:
confu(anomaly_result, test_category_label_binary)

array([[ 46015,    379],
       [ 14578, 250057]])

In [139]:
confu(attack_model.predict(testing_pca_value_attack[testing_index]), test_category_label[testing_index])

array([[  3893,    871,    151,    115],
       [   169, 228410,      5,   3943],
       [     8,    106,     18,    261],
       [    96,    466,     54,  11870]])

In [138]:
len(attack_model.predict(testing_pca_value_attack)), len(test_category_label[testing_index])

(311029, 250436)

In [140]:
attack_model.score(testing_pca_value_attack[testing_index],test_category_label[testing_index])

0.97506348927470488

In [143]:
sum(anomaly_result == test_category_label_binary) / float(len(anomaly_result))

0.95191123657279542

In [ ]:
1